Note: to view the plotly maps, use the nbviewer: https://nbviewer.jupyter.org/github/NickVance/Name-Country-and-Gender-Analysis/blob/master/Chicago%20Marathon%202018%20-%20Name%2C%20Country%2C%20and%20Gender%20Analysis.ipynb 

I analyzed the finishers data of the 2018 Chicago Marathon, looking for some insights into the breakdown of names, countries, and genders of the finishers. This method of analysis could easily be extended to examine similar things whenever you have a list of people along with their name, country, and gender.

I personally scraped the data from the official Chicago Marathon website the day after the event. Next, I did some preprocessing and cleaned the data, eventually ending up with a CSV file I called: Chicago_Marathon_2018_Results_Clean.txt . You are free to use that file to do your own analysis, but please link back to this page if you do so.

First, I imported the libraries I will need: pandas for data maniuplation and plotly for visualizations. You need your own plotly username and API code to use it, which you can insert into 'Your_Name' and 'Your_API_Code'

In [98]:
import pandas as pd

import plotly as py
import plotly.offline as offline

#plotly_username = 'Your_Name'
#plotly_api_key = 'Your_Api_Code'
plotly_username = 'NickVance'
plotly_api_key = 'B83jM1egoVPZgFvXyetY'
py.tools.set_credentials_file(username=plotly_username, api_key=plotly_api_key)

#Needed to view the plotly plots in a Jupyter Notebook
offline.init_notebook_mode(connected=True)

import warnings
#Ignore depracation warnings for this notebook
warnings.filterwarnings('ignore')

Plotly uses ISO-3 country codes, but the Chicago Marathon does not. I wasn't able to find out which system the Chicago Marathon actually uses, so I had to do these conversions on my own. There may be a faster way to do this, but this works. It requires the file IS03_Country_Codes.txt

In [99]:
ISO3_Codes = pd.DataFrame
ISO3_Codes = pd.read_table('ISO3_Country_Codes.txt', encoding='utf8')
ISO3_Codes = ISO3_Codes[['name', 'alpha-3']]

Codes_to_ISO3 = {}
Codes_to_ISO3.update(
        {'GER': 'DEU',
         'CRC': 'CRI',
         'TPE': 'TWN',
         'NED': 'NLD',
         'INA': 'IDN',
         'GUA': 'GTM',
         'POR': 'PRT',
         'CHI': 'CHL',
         'PHI': 'PHL',
         'SUI': 'CHE',
         'DEN': 'DNK',
         'SIN': 'SGP',
         'MAS': 'MYS',
         'PUR': 'PRI',
         'ESA': 'SLV',
         'HON': 'HND',
         'SLO': 'SVK',
         'PAR': 'PRY',
         'RSA': 'ZAF',
         "ALG":"DZA",
         "ARU":"ABW",
         "BAH":"BHS",
         "BAN":"BGD",
         "BAR":"BRB",
         "BER":"BMU",
         "BIZ":"BLZ",
         "BUL":"BGR",
         "CAY":"CYM",
         "CRO":"HRV",
         "GRE":"GRC",
         "IRI":"IRN",
         "KSA":"SAU",
         "KUW":"KWT",
         "LAT":"LVA",
         "LIB":"LBY",
         "MAW":"MWI",
         "MGL":"MNG",
         "MRI":"MRT",
         "NCA":"NIC",
         "NEP":"NPL",
         "NGR":"NGA",
         "PLE":"PSE",
         "TAN":"TZA",
         "TRI":"TTO",
         "UAE":"ARE",
         "URU":"URY",
         "VIE":"VNM",
         "ZAM":"ZMB",
         "ZIM":"ZWE",                      
         })

Now I import the finishers data from the CSV file I created and clean it up a bit.

In [100]:
Names = pd.DataFrame
Names = pd.read_table('Chicago_Marathon_2018_Results_Clean.txt', delimiter=',')

#Separate full name into first and last names
Names['Last Name'], Names['First Name']  = Names['Full Name'].str.split(', ', 1).str
Names['First Name'].fillna('none', inplace=True)
Names['Count'] = 1
del Names['Full Name']

#Change the country code to match the ISO-3 code, where needed
Names['Country'] = Names.apply(lambda row: Codes_to_ISO3[row['Country']] if row['Country'] in Codes_to_ISO3 else row['Country'], axis = 1)
Agg_Country = Names.groupby('Country', as_index=False).count()[['Country', 'Count']]

#This tests to make sure all of the country codes in my data are a valid ISO-3 code
Country_Code_Test = Agg_Country.copy()
Country_Code_Test = pd.merge(Country_Code_Test, ISO3_Codes, left_on = 'Country', right_on = 'alpha-3', how = 'left')
Country_Code_Test = Country_Code_Test[Country_Code_Test['alpha-3'].isnull()]
country_code_errors = len(Country_Code_Test)

if len(Country_Code_Test) == 0:
    print('No country code errors')
    
else:
        print('There are ' + str(country_code_errors) + ' incorrect country codes.')
        print(Country_Code_Test)

No country code errors


Here's what the dataframe Names looks like. It is the basis of all the analysis I do from this point on, so once you put your data in this form, you can use my code to quickly and easily run similar analysis on your data. Note that I never use the Last Name data, but it could also be interesting to look at.

In [101]:
print(Names.head())
print(' ')
print(Names.info())

  Country Gender Last Name       First Name  Count
0     GBR   male     Farah               Mo      1
1     ETH   male     Bayih  Mosinet Geremew      1
2     JPN   male     Osako           Suguru      1
3     KEN   male  Kipkemoi          Kenneth      1
4     USA   male      Rupp            Galen      1
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44573 entries, 0 to 44572
Data columns (total 5 columns):
Country       44573 non-null object
Gender        44573 non-null object
Last Name     44573 non-null object
First Name    44573 non-null object
Count         44573 non-null int64
dtypes: int64(1), object(4)
memory usage: 1.7+ MB
None


I now create a few other dataframes that I will use later

In [102]:
Agg_Gender = Names.groupby('Gender', as_index=False).count()[['Gender', 'Count']]
Agg_Name = Names.groupby('First Name', as_index=False).count()[['First Name', 'Count']]
Agg_Gender_by_Country = Names.groupby(['Country', 'Gender'], as_index=False).count()[['Country', 'Gender', 'Count']]
Agg_Name_Country = Names.groupby(['First Name', 'Country'], as_index=False).count()
Agg_Name_Gender = Names.groupby(['First Name', 'Gender'], as_index=False).count()[['First Name', 'Gender', 'Count']]
Agg_Name_Gender_by_Country = Names.groupby(['First Name', 'Gender', 'Country'], as_index=False).count()

#for the Agg_Name_MaxCountry dataframe, ties are broken by reverse alphabetical order
idx = Agg_Name_Country.groupby(['First Name'])['Count'].transform(max) == Agg_Name_Country['Count']
Agg_Name_MaxCountry = Agg_Name_Country[idx]
idx = Agg_Name_MaxCountry.groupby(['First Name'])['Country'].transform(max) == Agg_Name_MaxCountry['Country']
Agg_Name_MaxCountry = Agg_Name_MaxCountry[idx]

Now it's time to start the analysis. I first look at the total number of finishers, total number of countries represented, and which countries have the most runners. Almost 2/3 of the runners are from the United States, which makes sense considering the race takes place there and it would require a long flight to get to Chicago from anywhere outside the US except for a few parts of Canada.

In [103]:
#Calculating the total number of finishers
total_finishers = len(Names)
print('')
print('There are ' + str(total_finishers) + ' finishers.')

#Analyzing finishers by country
total_countries = len(Agg_Country)
print()
print('There are ' + str(total_countries) + ' countries represented.')

#Viewing the countries with the most representation
Agg_Country['Percent'] = 100 * round(Agg_Country['Count'] / total_finishers,3)
print()
print('Here are the ones with the most representation')
print(Agg_Country.sort_values('Count', ascending = False).head(5))


There are 44573 finishers.

There are 121 countries represented.

Here are the ones with the most representation
    Country  Count  Percent
114     USA  29129     65.4
71      MEX   2025      4.5
20      CAN   1411      3.2
43      GBR   1395      3.1
23      CHN   1227      2.8


I use this data to create a map which shows the number of participants by country. Hovering over each country gives the exact number of participants.

In [104]:
#Create a new dataframe to be used in the visualization of participants by country
By_Country = Agg_Country.copy()

data_country = [ dict(
        type = 'choropleth',
        locations = By_Country['Country'],
        z = By_Country['Count'],
        #Don't like the coloring - can I adjust it somehow to make it clearer?
        colorscale = [[0,"rgb(0, 200, 0)"],[1/10000,"rgb(0, 160, 0)"],[1/1000,"rgb(0, 120, 0)"],\
            [1/100,"rgb(0, 80, 0)"],[1/10,"rgb(0, 40, 0)"],[1,"rgb(0, 0, 0)"]],
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Number of Finishers'),
      ) ]

layout_country = dict(
    annotations=[
        dict(
            x=0.50,
            y=0.05,
            showarrow=False,
            xref='paper',
            yref='paper'
        )],
    title = '2018 Chicago Marathon <br> Finishers by Country',
    geo = dict(
        showframe = True,
        showcountries = True,
        countrywidth = .5,
        showcoastlines = True,
        coastlinewidth = .5,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data_country, layout=layout_country )

py.plotly.iplot(fig, validate=False, filename='chicago_marathon_finishers_by_country_map.html')

There were 19 countries that had exactly one finisher, and here's a map of them.

In [105]:
#Creating a dataframe that shows which countries only had one runner finish
One_Runner = Agg_Country[Agg_Country['Count'] == 1]
del One_Runner['Percent']
one_runner_count = len(One_Runner)
print()
print('There are ' + str(one_runner_count) + ' runners who had no fellow countrymen or countrywomen.')
print('Those countries are:')
print(One_Runner)

#Visualization of the One_Runner dataframe
data_country = [ dict(
        type = 'choropleth',
        locations = One_Runner['Country'],
        z = One_Runner['Count'],
        #Don't like the coloring - can I adjust it somehow to make it clearer?
        #colorscale = [[0,"rgb(0, 200, 0)"],[1/10000,"rgb(0, 160, 0)"],[1/1000,"rgb(0, 120, 0)"],\
         #   [1/100,"rgb(0, 80, 0)"],[1/10,"rgb(0, 40, 0)"],[1,"rgb(0, 0, 0)"]],
        #reversescale = False,
      ) ]

layout_country = dict(
    title = '2018 Chicago Marathon <br> Countries with Exactly One Finisher',
    showscale = False,
    geo = dict(
        showframe = True,
        showcountries = True,
        countrywidth = .5,
        showcoastlines = True,
        coastlinewidth = .5,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data_country, layout=layout_country )

py.plotly.iplot(fig, validate=False, filename='chicago_marathon_countries_with_one_finisher.html')



There are 19 runners who had no fellow countrymen or countrywomen.
Those countries are:
    Country  Count
0       ABW      1
2       ALB      1
3       AND      1
4       ARE      1
8       BDI      1
13      BIH      1
15      BLZ      1
25      CPV      1
35      DZA      1
59      JOR      1
72      MLI      1
73      MLT      1
75      MRT      1
88      PLW      1
96      SAU      1
97      SEN      1
100     SMR      1
105     TJK      1
115     UZB      1


There was some interesting data surrounding gender. Of countries with more than 50 finishers (44 countries), only one had more women than men. That was the US, with 52 percent of the finishers being female. On the other end of the spectrum, you had India (12 percent female) and Spain (15 percent female). Here's a map with all 44 of those countries and the exact numbers of male and female finishers.

It would be interesting to see how these percentages vary for other running races in other countries. Are there simply fewer female runners, do female runners tend to run marathon less often, or do they simply travel to other countries for marathons less often?

In [106]:
#Analyzing finishers by gender - overall
number_of_males = Agg_Gender.loc[Agg_Gender['Gender'] == 'male', 'Count'].iloc[0]
number_of_females = Agg_Gender.loc[Agg_Gender['Gender'] == 'female', 'Count'].iloc[0]
percent_male = number_of_males / total_finishers
print()
print('The finishers are ' + str(100*round(percent_male,2)) + '% male.')

#Looking at the gender division by country - create a dataframe that lists each country, the number and percent of male and female finishers
Gender_by_Country = Agg_Gender_by_Country.pivot(index = 'Country', columns = 'Gender')
Gender_by_Country.columns = ['Female', 'Male']
Gender_by_Country.fillna(0, inplace = True)
Gender_by_Country.reset_index(inplace=True)
Gender_by_Country['Total'] = Gender_by_Country['Male'] + Gender_by_Country['Female']
Gender_by_Country['Percent Male'] = 100*round(Gender_by_Country['Male'] / Gender_by_Country['Total'],3)
Gender_by_Country['Percent Female'] = 100*round(Gender_by_Country['Female'] / Gender_by_Country['Total'],3)
Gender_by_Country['Male'] = Gender_by_Country['Male'].astype('int')
Gender_by_Country['Female'] = Gender_by_Country['Female'].astype('int')
#Look at the countries with at least x finishers
Gender_by_Country = Gender_by_Country[Gender_by_Country['Total'] > 50]
Gender_by_Country['Map Text'] = Gender_by_Country['Male'].astype('str') + ' Males, ' + Gender_by_Country['Female'].astype('str') + ' Females'

#Visualizing the gender breakdown by country

data_gender = [dict(
        type = 'choropleth',
        locations = Gender_by_Country['Country'],
        z = Gender_by_Country['Percent Female'],
        text = Gender_by_Country['Map Text'],
        name = Gender_by_Country['Total'],
        colorscale = [[0,"rgb(200, 0, 0)"],[20,"rgb(160, 0, 0)"],[40,"rgb(120, 0, 0)"],\
            [60,"rgb(80, 0, 0)"],[80,"rgb(40, 0, 0)"],[100,"rgb(0, 0, 0)"]],
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            ticksuffix = ' %',
            title = 'Percentage of Female Runners'),
      )]

layout_gender = dict(
    annotations=[
        dict(
            x=0.50,
            y=0.05,
            showarrow=False,
            text='Note: Countries with less than 50 runners are ignored',
            xref='paper',
            yref='paper'
        )],
    title = '2018 Chicago Marathon <br> Percentage of Female Runners (by Country)',
    geo = dict(
        showframe = True,
        showcountries = True,
        countrywidth = .5,
        showcoastlines = True,
        coastlinewidth = .5,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict(data=data_gender, layout=layout_gender)

print(' ')
py.plotly.iplot(fig, validate=False, filename='chicago_marathon_gender_by_country_map')


The finishers are 54.0% male.
 
High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~NickVance/0 or inside your plot.ly account where it is named 'chicago_marathon_gender_by_country_map'


Then I moved on to analyzing names.

First, here are the most common names. Really interesting to me is the name Jennifer, which is the 4th most common name in the whole race, with the vast majority of Jennifer's coming from the US. It must not be a common name anywhere else.

In [107]:
#For each name, break down how many finishers are from the US and how many not from the US
By_Name = pd.merge(Agg_Name, Agg_Name_Country[Agg_Name_Country['Country'] == 'USA'], on='First Name', how='left')
del By_Name['Gender']
del By_Name['Country']
del By_Name['Last Name']
By_Name.columns = ['First Name', 'Total', 'USA']
By_Name.fillna(0, inplace=True)
By_Name['USA'] = By_Name['USA'].astype('int')
By_Name['Non-USA'] = By_Name['Total'] - By_Name['USA']
By_Name['Percent USA'] = 100 * By_Name['USA'] / By_Name['Total']

#Look at the most common names in total
By_Name.sort_values('Total', inplace=True, ascending=False)
print('')
print(By_Name.head())


     First Name  Total  USA  Non-USA  Percent USA
5983    Michael    661  577       84    87.291982
1851      David    466  344      122    73.819742
4015       John    435  383       52    88.045977
3783   Jennifer    348  328       20    94.252874
1776     Daniel    297  212       85    71.380471


Here are the names that were common among non-Americans. Carlos and Luis are extremely common around the world - mostly in Latin America - but not so common in the United States. The name David is interesting. The ratio of David's among American and non-American runners is quite stable, showing it's popularity both in the US and around the world.

In [108]:
#Look at the most common names among non-Americans
By_Name.sort_values('Non-USA', inplace=True, ascending=False)
print('')
print(By_Name.head())


     First Name  Total  USA  Non-USA  Percent USA
1851      David    466  344      122    73.819742
4101       Jose    184   82      102    44.565217
1146     Carlos    143   43      100    30.069930
5564      Maria    188   92       96    48.936170
5262       Luis    130   43       87    33.076923


I'm really interested in gender-neutral names, so I also looked at names that aren't exclusive to one gender. 

Andrea was an interesting one: among the runners in the Chicago Marathon, it was exclusively female for all countries except for one, Italy, where it was only a male name.

Jaime is also interested. It is a typical name in Spanish-speaking countries for males, and in the US it is an alternative spelling of Jamie, a name that is actually gender-neutral in the US.

In [109]:
#Add data on the gender-breakdown of names to the By_Name dataframe
By_Name = pd.merge(By_Name, Agg_Name_Gender[Agg_Name_Gender['Gender'] == 'male'], on = 'First Name', how = 'left')
By_Name.fillna(0, inplace=True)
del By_Name['Gender']
By_Name = pd.merge(By_Name, Agg_Name_Gender[Agg_Name_Gender['Gender'] == 'female'], on = 'First Name', how = 'left')
By_Name.fillna(0, inplace=True)
del By_Name['Gender']
By_Name.columns = ['First Name', 'Total', 'USA', 'Non-USA', 'Percent USA', 'Males', 'Females']
By_Name['Percent Male'] = By_Name['Males'] / (By_Name['Males'] + By_Name['Females'])

#Create a dataframe that has names with a substantial number from each gender
Gender_Neutral = By_Name[(By_Name['Percent Male'] > .2) & (By_Name['Percent Male'] < .8) & (By_Name['Total'] > 15)]
Gender_Neutral = Gender_Neutral.sort_values(by = ['Total'], ascending = False)
print(Gender_Neutral.head())

#Create individual dataframes for some names that are gender neutral to examine country-level effects
Andrea = Agg_Name_Gender_by_Country[Agg_Name_Gender_by_Country['First Name'] == 'Andrea']
Jamie = Agg_Name_Gender_by_Country[Agg_Name_Gender_by_Country['First Name'] == 'Jamie']
Jaime = Agg_Name_Gender_by_Country[Agg_Name_Gender_by_Country['First Name'] == 'Jaime']
Jan = Agg_Name_Gender_by_Country[Agg_Name_Gender_by_Country['First Name'] == 'Jan']
Jordan = Agg_Name_Gender_by_Country[Agg_Name_Gender_by_Country['First Name'] == 'Jordan']
Lee = Agg_Name_Gender_by_Country[Agg_Name_Gender_by_Country['First Name'] == 'Lee']

print('')
print(Andrea)
print('')
print(Jaime)

     First Name  Total  USA  Non-USA  Percent USA  Males  Females  \
214       Jamie     60   49       11    81.666667   17.0     43.0   
6311     Jordan     50   49        1    98.000000   31.0     19.0   
132       Jaime     44   28       16    63.636364   23.0     21.0   
584       Robin     32   27        5    84.375000    7.0     25.0   
406      Alexis     28   22        6    78.571429    8.0     20.0   

      Percent Male  
214       0.283333  
6311      0.620000  
132       0.522727  
584       0.218750  
406       0.285714  

    First Name  Gender Country  Last Name  Count
680     Andrea  female     ARG          2      2
681     Andrea  female     AUS          2      2
682     Andrea  female     AUT          1      1
683     Andrea  female     BRA          3      3
684     Andrea  female     CAN          5      5
685     Andrea  female     CHL          2      2
686     Andrea  female     COL         10     10
687     Andrea  female     CRI          3      3
688     Andrea  f

I was also interested in what names were the most "American": names that were fairly common among Americans but didn't have finishers from any other countries. The winner of that competition was Joshua - there were 84 Joshua's who completed the marathon, and no one from any other country.

In [110]:
#All-American names
All_American_Names = By_Name.copy()
All_American_Names = All_American_Names[All_American_Names['Non-USA'] == 0]
All_American_Names = All_American_Names[['First Name', 'Total', 'USA', 'Non-USA']]
All_American_Names.sort_values('Total', inplace=True, ascending=False)
print('')
print(All_American_Names.head())


     First Name  Total  USA  Non-USA
6560     Joshua     84   84        0
6533      Molly     60   60        0
6532    Kristen     58   58        0
6653    Brandon     56   56        0
6588      Jenna     52   52        0


I have some ideas for future analysis and projects. Feel free to take them and work on them yourself. If you do so, I'd love to see your work. We could also maybe work on something related to these together.
- Create an algorithm that guesses the country of origin based on information like gender, name, etc.
- Look at the data based on finishing time / place. Could look at how the country distribution changes based on how fast the runners are.
- Look at the same data for other marathons. Especially with gender ratios. Is a certain gender more or less likely to travel to different places? 
- Look at how the data changes over time for the same race. Are the gender ratios changing overall? Is it driven by real change, or merely demographic changes in the participants? For example, more Americans running and their gender ratio staying the same could lead to more than 46 percent of the runners overall being female.
- Examine people by the languages of their countries. What languages are the most common? 
- How to deal with double names? Look at the various double names and see what interesting stuff comes out.
- Create a visualization of the names used.

A few issues with the data I should let you know about:
- The country of origin is self-reporting. And I don't know if that is based on citizenship (and dual citizens exist), place of birth, residency, etc. For example, I am from the US but have German residency. Would I be listed under USA or Germany?
- This is finishers data, not participants data. It doesn't include people who started but didn't finish the race.
- I scraped the data on October 8, 2018 - the day after the race. The results were unofficial, so there might have been changes since then. However, for this kind of analysis, it's unlikely there would be significant changes.